In [77]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib auto

cv2.__version__

Using matplotlib backend: MacOSX


'3.1.0'

In [78]:
colList = []
def onclick(event):
    print('button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          (event.button, event.x, event.y, event.xdata, event.ydata))
    global colList
    colList.append(event.x)

def getPoint(x,y,img):
    temp = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return temp[x,y]

def getLineSegment(x1,x2,y,img):
    temp = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return temp[x1:x2,y]

In [79]:
frameOffset = 300
bufLen = 5

# grab and display a frame during run
cap = cv2.VideoCapture('test.mp4')
cap.set(cv2.CAP_PROP_POS_FRAMES,frameOffset)
ret,frame = cap.read()
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(frame)

# mouse click on columns to histogram
cid = fig.canvas.mpl_connect('button_press_event', onclick);


button=1, x=264, y=105, xdata=236.919355, ydata=413.693548
button=1, x=274, y=107, xdata=249.822581, ydata=411.112903
button=1, x=282, y=107, xdata=260.145161, ydata=411.112903
button=1, x=290, y=107, xdata=270.467742, ydata=411.112903
button=1, x=301, y=127, xdata=284.661290, ydata=385.306452


In [80]:
# get these with mouse clicks later
x1 = 300
x2 = 470

cap = cv2.VideoCapture('test.mp4')
cap.set(cv2.CAP_PROP_POS_FRAMES,0)

ret,frame = cap.read()
pixel = getLineSegment(x1,x2,colList[0],frame) 
buffer = np.tile(pixel, [2,numSubBuff,1])
subBuff = buffer[1,:,:].copy()

ret,frame = cap.read()
while ret:

    # create sub buffer
    for ii in range(0,numSubBuff):
        y = colList[ii]
        new = getLineSegment(x1,x2,y,frame) 
        subBuff[ii,:] = new    
    
    # append to main buffer
    buffer = np.append(buffer, np.expand_dims(subBuff, axis=0), axis=0)
    
    # read next frame
    ret,frame = cap.read()

cap.release()



True

In [81]:
# mask traces (remove noise) and aggregate
noiseThreshold = 90
buffer = buffer>noiseThreshold
out = np.sum(buffer, axis=1)
out = np.sum(out, axis=1)

# low pass filter to smooth
N=10
Fc=40
Fs=1600
import scipy.signal as sp

# use firwin with arbitrary Fs
h=sp.firwin( numtaps=N, cutoff=40, nyq=Fs/2)
y=sp.lfilter( h, 1.0, out) # 'x' is the time-series data you are filtering


# count number of positive inflextions points (first derivative zero / second neg)
dy = np.diff(y)
dy[dy>0] = 1
dy[dy<0] = -1
ddy = np.diff(dy)
ddy[ddy>0] = 0
np.count_nonzero(ddy)

1107